#Pretrained models

## Setup

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms, datasets, models
from torch.utils.data.sampler import SubsetRandomSampler, RandomSampler

import os
import random as rd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import zipfile
from google.colab import drive
drive.mount('/content/google')

zipfile.ZipFile('/content/google/MyDrive/Colab Notebooks/Udacity/deep_learning/landmark_images.zip').extractall()

In [ ]:
#@title Unnecessary
uneccessary = True
if not uneccessary:
    upload = True
    if upload:
        try:
            train_shapes = pd.read_csv('train_shapes.csv',header=0, index_col=0)
            test_shapes = pd.read_csv('test_shapes.csv',header=0, index_col=0)
        except FileNotFoundError:
            print ("Must upload files first.")

    else:
        #read train image file data:
        train_path = './landmark_images/train/'
        cat_path = [train_path + cat + '/' for cat in os.listdir(train_path)]
        jpg_list = [[cp + jpg for jpg in os.listdir(cp)] for cp in cat_path]

        train_images = []
        for jl in jpg_list:
            for jpg in jl:
                train_images.append(jpg)

        train_image_shapes = []
        for imfile in train_images:
            train_image_shapes.append(cv2.imread(imfile).shape)
        heights = [shp[0] for shp in train_image_shapes]    
        widths = [shp[1] for shp in train_image_shapes]
        
        train_images_df = pd.DataFrame(columns=['filepath','height','width'])
        train_images_df['filepath'] = train_images
        train_images_df['height'] = heights
        train_images_df['width'] = widths
        train_images_df.to_csv('train_shapes.csv')
        
        #read test image file data:
        test_path = './landmark_images/test/'
        cat_path = [test_path + cat + '/' for cat in os.listdir(test_path)]
        jpg_list = [[cp + jpg for jpg in os.listdir(cp)] for cp in cat_path]

        test_images = []
        for jl in jpg_list:
            for jpg in jl:
                test_images.append(jpg)

        test_image_shapes = []
        for imfile in test_images:
            test_image_shapes.append(cv2.imread(imfile).shape)
        heights = [shp[0] for shp in test_image_shapes]    
        widths = [shp[1] for shp in test_image_shapes] 

        test_images_df = pd.DataFrame(columns=['filepath','height','width'])
        test_images_df['filepath'] = test_images
        test_images_df['height'] = heights
        test_images_df['width'] = widths
        test_images_df.to_csv('test_shapes.csv')

        #cleanup
        del train_images, train_image_shapes, train_path
        del cat_path, jpg_list, widths, heights
        del test_images, test_image_shapes, test_path
        #del train_images_df, test_images_df

    def quick_label(row):
        label = row['filepath'][-40:-20] + "\nWidth: " + str(row['width']) + "  Height: " + str(row['height'])
        return label

    choices = rd.sample(list(train_shapes.index), 4)
    paths = [(train_shapes.iloc[s])['filepath'] for s in choices]
    labels = [quick_label(train_shapes.iloc[s]) for s in choices]
    imgs_bgr = [plt.imread(p) for p in paths]
    imgs_rgb = [bgr2rgb(im) for im in imgs_bgr] 

    fig, axes = plt.subplots(figsize=(18,16), nrows=2, ncols=2)
    axes = axes.reshape(4)
    for ii, im in enumerate(imgs_rgb):
        ax0 = axes[ii] 
        ax0.set_title(labels[ii])
        ax0.imshow(im)


In [ ]:
#@ title Image transpose
def bgr2rgb(im_bgr):
    # because cv2.imread() returns BGR image format
    im_rgb = im_bgr.copy()
    im_rgb[:,:,0] = im_bgr[:,:,2]
    im_rgb[:,:,1] = im_bgr[:,:,1]
    im_rgb[:,:,2] = im_bgr[:,:,0]
    return im_rgb

In [ ]:
(3997//25, 3997%25), (999//25, 999%25), (1250//25, 1250%25)

In [ ]:
train_shapes = pd.read_csv('train_shapes.csv',header=0, index_col=0)
test_shapes = pd.read_csv('test_shapes.csv',header=0, index_col=0)
# Cat nums to labels
classvals = list(set([(str(pair[0]),pair[1]) for pair in test_shapes[['labnum','label']].values]))
num2class = {int(x):y for x,y in classvals}        

In [ ]:
# image directory location
data_dir = '/content/landmark_images'

batch_size = 25

# set target height and width for all images
H = 300
W = 400
h0 = 450
w0 = 600
h1 = 600
w1 = 800

train_transforms = transforms.Compose( [transforms.Resize((h1, w1)),
                                        transforms.RandomCrop((h0,w0)),
                                        transforms.RandomRotation(20, 
                                                                  fill=np.random.randint(100,201)),
                                        transforms.CenterCrop((H,W)),
                                        transforms.ToTensor()])

test_transforms = transforms.Compose([ transforms.Resize((H, W)),
                                       transforms.ToTensor()])

train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(0.2 * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)
test_sampler = RandomSampler(test_data)

trainloader = torch.utils.data.DataLoader(train_data, 
                                          batch_size=batch_size,
                                          sampler=train_sampler)

validloader = torch.utils.data.DataLoader(train_data, 
                                          batch_size=batch_size,
                                          sampler=valid_sampler)

testloader = torch.utils.data.DataLoader(test_data, 
                                         batch_size=batch_size,
                                         sampler=test_sampler)

loaders_scratch = {'train': trainloader, 'valid': validloader, 'test': testloader}

---
## Load VGG16 as revgg

To define a model for training we'll follow these steps:
1. Load in a pre-trained VGG16 model
2. "Freeze" all the parameters, so the net acts as a fixed feature extractor 
3. Remove the last layer
4. Replace the last layer with a linear classifier of our own

**Freezing simply means that the parameters in the pre-trained model will *not* change during training.**

In [ ]:
# Load the pretrained models from pytorch
revgg = models.vgg16(pretrained=True)

# print out the model structure
print(revgg)

In [ ]:
# Freeze training for all "features" layers
for param in revgg.features.parameters():
    param.requires_grad = False
    print(param.size(), param.requires_grad) 
    
#for clsfr in revgg.classifier:
#    for prmtr in clsfr.parameters():
#        prmtr.requires_grad = True
#        print(prmtr.size(), prmtr.requires_grad) 

---
### Final Classifier Layer

Once you have the pre-trained feature extractor, you just need to modify and/or add to the final, fully-connected classifier layers. In this case, we suggest that you repace the last layer in the vgg classifier group of layers. 
> This layer should see as input the number of features produced by the portion of the network that you are not changing, and produce an appropriate number of outputs for the flower classification task.

You can access any layer in a pretrained network by name and (sometimes) number, i.e. `vgg16.classifier[6]` is the sixth layer in a group of layers named "classifier".

In [ ]:
use_cuda = torch.cuda.is_available()

In [ ]:
n_inputs = revgg.classifier[6].in_features

# new layers automatically have requires_grad = True
fork_layer = nn.Linear(n_inputs, 50)
#last_layer = nn.Linear(50*len(classes), len(classes))

revgg.classifier[6] = fork_layer

# if GPU is available, move the model to GPU
if use_cuda:
    revgg.cuda()

# check to see that your last layer produces the expected number of outputs
print(revgg)

In [ ]:
# Check for all layers if trainable on training
for param in revgg.features.parameters():
    print(param.size(), param.requires_grad) 
print("\n")
for param in revgg.classifier.parameters():
    print(param.size(), param.requires_grad) 

In [ ]:
#import torch.optim as optim

# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.001
vgg_optimizer = optim.SGD(revgg.classifier.parameters(), lr=0.005)

for p in revgg.classifier[6].parameters():
    n = p.size(0)
    torch.nn.init.normal_(p, mean=0.5, std=0.5)#std=1.0/(n**0.5))

---
### Training



In [ ]:
#@ title retrain def

def retrain(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns retrained pretrained-model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    no_loss = 0

    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0

        ###################
        # train the model #
        ###################
        # set the module to training mode
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()*data.size(0)       
        train_loss = train_loss/len(loaders['train'].dataset) 

        ######################    
        # validate the model #
        ######################
        # set the model to evaluation mode
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            output = model(data)
            loss = criterion(output, target)
            valid_loss += loss.item()*data.size(0)      
        valid_loss = valid_loss/len(loaders['valid'].dataset)

        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
                                                                    epoch, 
                                                                    train_loss,
                                                                    valid_loss
                                                                    ))
        if valid_loss < valid_loss_min:
            print('* Loss decrease *'.format(valid_loss_min, valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
            no_loss = 0
        else:
            no_loss += 1
        
        if no_loss >= 3:
            print("=== Loss is no longer decreasing. Ending training. ===")
            break
    return model

In [ ]:
n_epochs = 20
revgg = retrain(n_epochs, loaders_scratch, 
                revgg, vgg_optimizer, criterion, 
                use_cuda, 'revgg_last3.pt')

---
### Testing

In [ ]:

#batch_size=batch_size//2
#testloader = torch.utils.data.DataLoader(test_data, 
#                                         batch_size=batch_size,
#                                         sampler=test_sampler)

# track test loss 
test_loss = 0.0
class_correct = list(0. for i in range(50))
class_total = list(0. for i in range(50))

revgg.eval() # eval mode

# iterate over test data
for data, target in testloader:
    if use_cuda:
        data, target = data.cuda(), target.cuda()
    output = revgg(data)
    # calculate the batch loss
    loss = criterion(output, target)
    # update  test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)    
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy())\
              if not use_cuda\
              else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
    for i in range(data.size(0)):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# calculate avg test loss
test_loss = test_loss/len(testloader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(50):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            num2class[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (num2class[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

### Visualize Sample Test Results

In [ ]:
# obtain one batch of test images
try:
    images, labels = dataiter.next()
except NameError:
    dataiter = iter(testloader)
    images, labels = dataiter.next()
images.numpy()

# move model inputs to cuda, if GPU available
if use_cuda:
    images = images.cuda()

# get sample outputs
output = revgg(images)
# convert output probabilities to predicted class
_, preds_tensor = torch.max( output, 1)
preds = np.squeeze(preds_tensor.numpy()) if not use_cuda else np.squeeze(preds_tensor.cpu().numpy())

# plot the images in the batch, along with predicted and true labels
fig = plt.figure(figsize=(18, 20))
for idx in np.arange(12):
    ax = fig.add_subplot(4, 12/4, idx+1, xticks=[], yticks=[])
    plt.imshow(np.transpose(images.cpu()[idx], (1, 2, 0)))
    ax.set_title("{}\n({})".format(num2class[preds[idx]], num2class[labels[idx].item()]),
                                  color=("blue" if preds[idx]==labels[idx].item() else "red"))

---
## Load RESNET as reres

In [ ]:
# Load the pretrained models from pytorch
reres = models.resnet50(pretrained=True)

# print out the model structure
print(reres)

In [ ]:
for fcp in reres.fc.parameters():
    print(p.requires_grad)

In [ ]:
# Freeze training for all convolutional layers
for param in reres.parameters():
    param.requires_grad = False
    print(param.size(), param.requires_grad) 

# Don't freeze training for the linear layers 
#for clsfr in reres.classifier:
#    for prmtr in clsfr.parameters():
#        prmtr.requires_grad = True
#        print(prmtr.size(), prmtr.requires_grad) 

In [ ]:
n_inputs = reres.fc.in_features

# new layers automatically have requires_grad = True
fork_layer = nn.Linear(n_inputs, 50))

reres.fc = fork_layer

# if GPU is available, move the model to GPU
if use_cuda:
    reres.cuda()

# check to see that your last layer produces the expected number of outputs
print(reres)

In [ ]:
# Check for all layers if trainable on training
for param in reres.features.parameters():
    print(param.size(), param.requires_grad) 
print("\n")
for param in reres.classifier.parameters():
    print(param.size(), param.requires_grad) 

In [ ]:
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# Only linear classifier layers go to optimizer
res_optimizer = optim.SGD(reres.classifier.parameters(), lr=0.01)

# Initialize weights for classifier layers
for p in reres.classifier[6].parameters():
    n = p.size(0)
    torch.nn.init.normal_(p, mean=0.2, std=1.0/(n**0.5))

In [ ]:
use_cuda = torch.cuda.is_available()

---
### Train



In [ ]:
#@ title retrain def

def retrain(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns retrained pretrained-model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    no_loss = 0

    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0

        ###################
        # train the model #
        ###################
        # set the module to training mode
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()*data.size(0)       
        train_loss = train_loss/len(loaders['train'].dataset) 

        ######################    
        # validate the model #
        ######################
        # set the model to evaluation mode
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            output = model(data)
            loss = criterion(output, target)
            valid_loss += loss.item()*data.size(0)      
        valid_loss = valid_loss/len(loaders['valid'].dataset)

        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
                                                                    epoch, 
                                                                    train_loss,
                                                                    valid_loss))
        if valid_loss < valid_loss_min:
            print('* Loss decrease *'.format(valid_loss_min, valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
        else:
            no_loss += 1
        
        if no_loss >= 3:
            print("=== Loss is no longer decreasing. Ending training. ===")
            break

    return model

In [ ]:
reres = retrain(n_epochs, loaders_scratch, 
                reres, vgg_optimizer, criterion, 
                use_cuda, 'reres_last3.pt')

---
### Test

In [ ]:

batch_size=15
testloader = torch.utils.data.DataLoader(test_data, 
                                         batch_size=batch_size,
                                         sampler=test_sampler)

# track test loss 
test_loss = 0.0
class_correct = list(0. for i in range(50))
class_total = list(0. for i in range(50))

reres.eval() # eval mode

# iterate over test data
for data, target in testloader:
    if use_cuda:
        data, target = data.cuda(), target.cuda()
    output = reres(data)
    # calculate the batch loss
    loss = criterion(output, target)
    # update  test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)    
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy())\
              if not use_cuda\
              else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
    for i in range(data.size(0)):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# calculate avg test loss
test_loss = test_loss/len(testloader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(50):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            num2class[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (num2class[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

### Visualize

In [ ]:
# obtain one batch of test images
try:
    images, labels = dataiter.next()
except NameError:
    dataiter = iter(validloader)
    images, labels = dataiter.next()
images.numpy()

# move model inputs to cuda, if GPU available
if use_cuda:
    images = images.cuda()

# get sample outputs
output = reres(images)
# convert output probabilities to predicted class
_, preds_tensor = torch.max( output, 1)
preds = np.squeeze(preds_tensor.numpy()) if not use_cuda else np.squeeze(preds_tensor.cpu().numpy())

# plot the images in the batch, along with predicted and true labels
fig = plt.figure(figsize=(18, 25))
for idx in np.arange(15):
    ax = fig.add_subplot(5, 15/5, idx+1, xticks=[], yticks=[])
    plt.imshow(np.transpose(images.cpu()[idx], (1, 2, 0)))
    ax.set_title("{}\n({})".format(num2class[preds[idx]], num2class[labels[idx].item()]),
                                  color=("blue" if preds[idx]==labels[idx].item() else "red"))

## Alexnet

In [ ]:
anet = models.alexnet(pretrained=True)
print (anet)

# Keras model

## load data and modules

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import os
import random as rd
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import CategoricalCrossentropy as cat_entropy
from tensorflow.keras.losses import SparseCategoricalCrossentropy as sparse_cat_entropy

In [ ]:
import zipfile
from google.colab import drive
drive.mount('/content/drive')

zipfile.ZipFile('/content/drive/MyDrive/Colab Notebooks/Udacity/deep_learning/landmark_images.zip').extractall()

In [ ]:
try:
    train_shapes = pd.read_csv('train_shapes.csv',header=0, index_col=0)
    test_shapes = pd.read_csv('test_shapes.csv',header=0, index_col=0)
except FileNotFoundError:
    print ("Must upload train_shapes.csv and test_shapes.csv first.")

if False:
    
    #read train image file data:
    train_path = './landmark_images/train/'
    cat_path = [train_path + cat + '/' for cat in os.listdir(train_path)]
    cat_list = [cat for cat in os.listdir(train_path)]
    cats, train_images = [], []
    for c in cat_list:
        cp = str(train_path + c + '/') 
        for imfile in os.listdir(cp):
            train_images.append(str(cp + imfile))
            cats.append(c)

    train_image_shapes = []
    for imfile in train_images:
        train_image_shapes.append(plt.imread(imfile).shape)
    heights = [shp[0] for shp in train_image_shapes]    
    widths = [shp[1] for shp in train_image_shapes]

    train_images_df = pd.DataFrame(columns=['labnum','label',
                                            'height','width',
                                            'filepath'])
    train_images_df['labnum'] = [int(c[:2]) for c in cats]
    train_images_df['label'] = [c[3:] for c in cats]
    train_images_df['height'] = heights
    train_images_df['width'] = widths
    train_images_df['filepath'] = train_images

    #read test image file data:
    test_path = './landmark_images/test/'
    cat_path = [test_path + cat + '/' for cat in os.listdir(test_path)]
    cat_list = [cat for cat in os.listdir(test_path)]
    cats, test_images = [], []
    for c in cat_list:
        cp = str(test_path + c + '/') 
        for imfile in os.listdir(cp):
            test_images.append(str(cp + imfile))
            cats.append(c)

    test_image_shapes = []
    for imfile in test_images:
        test_image_shapes.append(plt.imread(imfile).shape)
    heights = [shp[0] for shp in test_image_shapes]    
    widths = [shp[1] for shp in test_image_shapes]

    test_images_df = pd.DataFrame(columns=['labnum','label',
                                            'height','width',
                                            'filepath'])
    test_images_df['labnum'] = [int(c[:2]) for c in cats]
    test_images_df['label'] = [c[3:] for c in cats]
    test_images_df['height'] = heights
    test_images_df['width'] = widths
    test_images_df['filepath'] = test_images

    #save dfs
    train_images_df.to_csv('train_shapes.csv')
    test_images_df.to_csv('test_shapes.csv')

    #cleanup
    del train_images, train_image_shapes, train_path
    del test_images, test_image_shapes, test_path
    del cats, cat_list, cat_path, widths, heights

In [ ]:
## Only want exaclty 600x800 images
testable_df = test_shapes.loc[lambda df: df['width'] == 800].loc[lambda df: df['height'] == 600]
trainable_df = train_shapes.loc[lambda df: df['width'] == 800].loc[lambda df: df['height'] == 600]

## Make class dict
classvals = list(set([(str(pair[0]),pair[1]) for pair in test_shapes[['labnum','label']].values]))
num2class = {int(x):y for x,y in classvals}

In [ ]:
f = plt.figure(num=1, figsize=(20,20))
for i in range(3):
    randix = rd.choice(list(testable_df.index))
    img = plt.imread(test_shapes.iloc[randix]['filepath'])/255.
    tar = test_shapes.iloc[randix]['labnum']

    ax = f.add_subplot(1,3,i+1)
    ax.set_title("Class #"+str(tar)+" "+num2class[tar])
    ax.imshow(img)

## experimental models

In [ ]:
#@title Experimental Keras Model

input_layer = layers.Input(shape=(600,800,3))
z = layers.Conv2D(10, 3, 
                  strides=2,
                  activation='relu',
                  #kernel_initializer='uniform',
                  use_bias=True,
                  padding='same')(input_layer)

y = layers.Conv2D(10, 3, 
                  strides=2,
                  activation='relu',
                  #kernel_initializer='normal',
                  use_bias=True,
                  padding='same')(z)

x = layers.Conv2D(20, 7, 
                  strides=1,
                  activation='relu',
                  use_bias=True,
                  padding='valid')(y)
x = layers.MaxPooling2D(2)(x)

x = layers.Conv2D(30, 5, 
                  strides=1,
                  activation='relu',
                  use_bias=True,
                  padding='valid')(x)
x = layers.MaxPooling2D(2)(x)

x = layers.Conv2D(40, 3, 
                  strides=1,
                  activation='relu',
                  use_bias=True,
                  padding='valid')(x)
x = layers.MaxPooling2D(2)(x)

x = layers.Conv2D(60, 3, 
                  strides=1,
                  activation='relu',
                  use_bias=True,
                  padding='valid')(x)
x = layers.MaxPooling2D(2)(x)

x = layers.Flatten()(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(1000, activation='relu')(x)
x = layers.Dropout(0.2)(x)
output_layer = layers.Dense(10, activation='softmax')(x)

shrinkX = Model(input_layer, y)
modelX = Model(input_layer, output_layer)
modelX.summary()

In [ ]:
def scale(a):
    return (a-a.min())/(a.max()-a.min())

In [ ]:
f = plt.figure(num=2,figsize=(20,5))
sp1 = f.add_subplot(1,4,1)
sp2 = f.add_subplot(1,4,2)
sp3 = f.add_subplot(1,4,3)
sp4 = f.add_subplot(1,4,4)

randix = rd.choice(list(trainable_df.index))
img = plt.imread(train_shapes.iloc[randix]['filepath'])/255.
tar = train_shapes.iloc[randix]['labnum']

out_img = shrinkX(np.expand_dims(img,0))
out1 = out_img.numpy()[0,:,:,0:3]
out2 = out_img.numpy()[0,:,:,5:8]
out3 = out_img.numpy()[0,:,:,7:10]


sp1.set_title("Class #"+str(tar)+" "+num2class[tar])
sp1.imshow(scale(img))
sp2.set_title("Model Output One")
sp2.imshow(scale(out1))
sp3.set_title("Model Output Two")
sp3.imshow(scale(out2))
sp4.set_title("Model Output Three")
sp4.imshow(scale(out3))

In [ ]:
print("min\t max\t avg\t std")
for i in range(10):  
    print("{:.4f}\t {:.4f}\t {:.4f}\t {:.4f}".format(   out_img.numpy()[0,:,:,i].min(), 
                                                        out_img.numpy()[0,:,:,i].max(),
                                                        out_img.numpy()[0,:,:,i].mean(),
                                                        out_img.numpy()[0,:,:,i].std()) )

#### build and train fnxs

In [ ]:
#traintupleDG, testupleDG = tf.keras.datasets.mnist.load_data()

In [ ]:
## Initiate hyperparameters
epochs = 5
learning_rate = 0.01
batch_size = 40
validation_batch_size = 20

## Construct datasets
x_train = np.array([plt.imread(fp) for fp in trainable_df["filepath"]])/255.
y_train = float(trainable_df[["labnum"]].values.squeeze())

x_test = np.array([plt.imread(fp) for fp in testable_df["filepath"]])/255.
y_test = float(testable_df[["labnum"]].values.squeeze())

traintuple = (x_train[:4], y_train[:4])
testtuple = (x_test[:4], y_test[:4])

In [ ]:
del x_test
del y_test
del x_train
del y_train
del traintuple
del testtuple

In [ ]:
#@title Define functions for building, training, compiling and evaluating models

def build_model(verbose=False):
    '''Builds a model and returns it uncompiled

    '''
    input_layer = layers.Input(shape=(600,800,3))
    z = layers.Conv2D(10, 3, 
                    strides=2,
                    activation='relu',
                    #kernel_initializer='uniform',
                    use_bias=True,
                    padding='same')(input_layer)

    y = layers.Conv2D(10, 3, 
                    strides=2,
                    activation='relu',
                    #kernel_initializer='normal',
                    use_bias=True,
                    padding='same')(z)

    x = layers.Conv2D(20, 7, 
                    strides=1,
                    activation='relu',
                    use_bias=True,
                    padding='valid')(y)
    x = layers.MaxPooling2D(2)(x)

    x = layers.Conv2D(30, 5, 
                    strides=1,
                    activation='relu',
                    use_bias=True,
                    padding='valid')(x)
    x = layers.MaxPooling2D(2)(x)

    x = layers.Conv2D(40, 3, 
                    strides=1,
                    activation='relu',
                    use_bias=True,
                    padding='valid')(x)
    x = layers.MaxPooling2D(2)(x)

    x = layers.Conv2D(60, 3, 
                    strides=1,
                    activation='relu',
                    use_bias=True,
                    padding='valid')(x)
    x = layers.MaxPooling2D(2)(x)

    x = layers.Flatten()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(1000, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    output_layer = layers.Dense(10, activation='softmax')(x)

    return Model(input_layer, output_layer)

## Same training loop for all models
def train(model, traintuple, valtuple, epochs=epochs):
    '''Train a model on the given sets of data
        Params: the given model,
                the train data as a tuple of x,y,
                the test data as a tuple of x,y
        Returns: a dictionary of metric values after each epoch of training
    '''
    (x_train, y_train) = traintuple    
    history = model.fit(x=x_train,
                        y=y_train,
                        batch_size=batch_size,
                        validation_data=valtuple,
                        validation_batch_size=validation_batch_size,
                        epochs=epochs,  
                        verbose=1)   
    return history.history
print("Loaded function train(model, traintuple, testuple, epochs=epochs)")

## All models are compiled the same
def compile_model(model):    
    model.compile(  loss=sparse_cat_entropy,
                    optimizer=SGD(learning_rate=learning_rate),                    
                    metrics=['acc'])
    print ("Compiled model", model.name)
print("loaded function compile_model(model)")

In [ ]:
def build_mini(verbose=False):
    '''Builds a minimal experimental model and returns it uncompiled

    '''
    input_layer = layers.Input(shape=(600,800,3))
    x = layers.Conv2D(10, 3, 
                    strides=2,
                    activation='relu',
                    use_bias=True,
                    padding='same')(input_layer)

    x = layers.MaxPooling2D(3)(x)
    x = layers.Flatten()(x)
    x = layers.Dropout(0.1)(x)
    output_layer = layers.Dense(50, activation='softmax')(x)

    return Model(input_layer, output_layer, name='minimodel')

In [ ]:
def qtrain(model, traintuple, valtuple, epochs=epochs):
    '''Train a model on the given sets of data
        Params: the given model,
                the train data as a tuple of x,y,
                the test data as a tuple of x,y
        Returns: a dictionary of metric values after each epoch of training
    '''
    (x_train, y_train) = traintuple    
    history = model.fit(x=x_train,
                        y=y_train,
                        batch_size=batch_size,
                        validation_data=valtuple,
                        validation_batch_size=validation_batch_size,
                        epochs=epochs,  
                        verbose=1)   
    return history.history

def qcompile_model(model):    
    model.compile(  loss=sparse_cat_entropy,
                    optimizer=SGD(learning_rate=learning_rate))#,                    
                    #metrics=['acc'])
    print ("Compiled model", model.name)

In [ ]:
epochs = 1
learning_rate = 0.01
batch_size = 2
validation_batch_size = 2

In [ ]:
### Build
mm = build_mini()

### Compile
qcompile_model(mm)


In [ ]:
######### Print out summary table
print(mm.summary(),"\n")

######## Plot model diagrams
tf.keras.utils.plot_model(mm, 
                          show_layer_names=True, 
                          show_shapes=True, 
                          to_file="mm.png")

In [ ]:
### Train
train_stats = qtrain(mm, traintuple, testtuple, epochs=epochs)

In [ ]:
randix = rd.choice(list(trainable_df.index))
img = plt.imread(train_shapes.iloc[randix]['filepath'])/255.
tar = train_shapes.iloc[randix]['labnum']

out = np.argmax(mm(np.expand_dims(img,0)).numpy().squeeze())
print("target",tar,"\toutput", out)

In [ ]:
traintuple[1].dtype